# Library

In [1]:
!pip install keybert
!pip install pyforest

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 989.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 28.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel, pipeline
from tqdm import tqdm
import torch, pyforest, re
import numpy as np
import math
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Download data stopword jika belum terunduh
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

# Load Data

In [ ]:
df = pd.read_excel('/content/Dataset_Anotasi.xlsx')
df = df.iloc[:100]

<IPython.core.display.Javascript object>

# KeyBERT

In [3]:
# Model KeyBERT
sentence_model = SentenceTransformer("denaya/indoSBERT-large")

# Buat objek KeyBERT
kw_model = KeyBERT(model=sentence_model)

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.23k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/709k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

In [ ]:
df['keyword_sihab'] = df.apply(lambda x: [x['keyword1_sihab'], x['keyword2_sihab'], x['keyword3_sihab'], x['keyword4_sihab'], x['keyword5_sihab']], axis=1)
df['keyword_naufal'] = df.apply(lambda x: [x['keyword1_naufal'], x['keyword2_naufal'], x['keyword3_naufal'], x['keyword4_naufal'], x['keyword5_naufal']], axis=1)

In [ ]:
df_keyword = df[['keluhan_baku', 'keyword_sihab', 'keyword_naufal']]

In [ ]:
# Ekstrak keyword untuk semua teks
df_keyword["keyword_keybert"] = df_keyword["keluhan_baku"].apply(lambda x: [kw[0] for kw in kw_model.extract_keywords(x, top_n=10)])
df_keyword.head()

,keluhan_baku,keyword_sihab,keyword_naufal,keyword_keybert
0,proyek pembangunan rumah tinggal membakar samp...,"[asap, mengganggu, membakar, mencemari, sampah]","[limbah, mencemari, mengganggu, tindaklanjuti,...","[mengganggu, asap, mencemari, limbah, sampah, ..."
1,ass wr wb selamat siang yth kepala dinas sumbe...,"[pavingisasi, hujan, banjir, aspal, tidak layak]","[memprihatinkan, banjir, rusak, tidak layak, m...","[banjir, pembangunan, ardinanta, pavingisasi, ..."
2,1 kartu keluarga nama yang terdaftar mbr des 2...,"[nama, tidak terdaftar, mbr, KK, nan]","[tidak terdaftar, data MBR, cek, menanyakan, k...","[2021, keluarga, mbr, kartu, semua, terdaftar,..."
3,pandemi maret 2020 sudah mengurus mbr 4 kali b...,"[pandemi, mengurus, mbr, tidak berhasil, nan]","[tidak berhasil, pandemi, mengurus, MBR, berka...","[mbr, subhakti, yatim, 2020, pandemi, sudah, m..."
4,mohon dibantu pembersihan waduk kebraon sebera...,"[pembersihan, waduk, sampah, numpuk, nan]","[mohon, dibantu, pembersihan, sampah, numpuk]","[karangpilang, waduk, eceng, sampah, numpuk, k..."


# TextRank

In [ ]:
# Fungsi untuk menghitung skor TextRank dan menyimpan kata kunci teratas
def calculate_textrank_keywords(text):
    # Tokenisasi teks
    tokens = word_tokenize(text)
    # Hapus tanda baca, angka, dan stopword
    tokens = [word for word in tokens if word.isalpha()]  # Hapus angka
    stop_words = set(stopwords.words('indonesian'))  # Daftar stopword bahasa Indonesia
    tokens = [word for word in tokens if word not in stop_words]  # Hapus stopword

    vocabulary = list(set(tokens))

    vocab_len = len(vocabulary)

    # Matriks weighted edge
    weighted_edge = np.zeros((vocab_len, vocab_len), dtype=np.float32)
    score = np.zeros((vocab_len), dtype=np.float32)
    window_size = 3
    covered_coocurrences = []

    for i in range(0, vocab_len):
        score[i] = 1
        for j in range(0, vocab_len):
            if j == i:
                weighted_edge[i][j] = 0
            else:
                for window_start in range(0, (len(tokens) - window_size)):
                    window_end = window_start + window_size
                    window = tokens[window_start:window_end]
                    if (vocabulary[i] in window) and (vocabulary[j] in window):
                        index_of_i = window_start + window.index(vocabulary[i])
                        index_of_j = window_start + window.index(vocabulary[j])
                        if [index_of_i, index_of_j] not in covered_coocurrences:
                            weighted_edge[i][j] += 1 / math.fabs(index_of_i - index_of_j)
                            covered_coocurrences.append([index_of_i, index_of_j])

    inout = np.zeros((vocab_len), dtype=np.float32)

    for i in range(0, vocab_len):
        for j in range(0, vocab_len):
            inout[i] += weighted_edge[i][j]

    MAX_ITERATIONS = 50
    d = 0.85
    threshold = 0.0001
    for iter in range(0, MAX_ITERATIONS):
        prev_score = np.copy(score)
        for i in range(0, vocab_len):
            summation = 0
            for j in range(0, vocab_len):
                if weighted_edge[i][j] != 0:
                    summation += (weighted_edge[i][j] / inout[j]) * score[j]
            score[i] = (1 - d) + d * (summation)
        if np.sum(np.fabs(prev_score - score)) <= threshold:
            break

    # Menyimpan 5 kata kunci teratas dengan skor tertinggi
    top_keywords = [vocabulary[i] for i in np.argsort(score)[-5:][::-1]]
    return top_keywords

# Terapkan fungsi pada setiap baris teks dalam dataframe
df_keyword['keyword_textrank'] = df_keyword['keluhan_baku'].apply(calculate_textrank_keywords)

# Tampilkan hasil
df_keyword.head()

,keluhan_baku,keyword_sihab,keyword_naufal,keyword_keybert,keyword_textrank
0,proyek pembangunan rumah tinggal membakar samp...,"[asap, mengganggu, membakar, mencemari, sampah]","[limbah, mencemari, mengganggu, tindaklanjuti,...","[mengganggu, asap, mencemari, limbah, sampah, ...","[rumah, pemukiman, tinggal, mengganggu, membakar]"
1,ass wr wb selamat siang yth kepala dinas sumbe...,"[pavingisasi, hujan, banjir, aspal, tidak layak]","[memprihatinkan, banjir, rusak, tidak layak, m...","[banjir, pembangunan, ardinanta, pavingisasi, ...","[wilayah, tegalsari, kepala, dinas, kelurahan]"
2,1 kartu keluarga nama yang terdaftar mbr des 2...,"[nama, tidak terdaftar, mbr, KK, nan]","[tidak terdaftar, data MBR, cek, menanyakan, k...","[2021, keluarga, mbr, kartu, semua, terdaftar,...","[kartu, keluarga, mbr, terdaftar, des]"
3,pandemi maret 2020 sudah mengurus mbr 4 kali b...,"[pandemi, mengurus, mbr, tidak berhasil, nan]","[tidak berhasil, pandemi, mengurus, MBR, berka...","[mbr, subhakti, yatim, 2020, pandemi, sudah, m...","[nama, mengurus, berhasil, mbr, kali]"
4,mohon dibantu pembersihan waduk kebraon sebera...,"[pembersihan, waduk, sampah, numpuk, nan]","[mohon, dibantu, pembersihan, sampah, numpuk]","[karangpilang, waduk, eceng, sampah, numpuk, k...","[kebraon, numpuk, kertas, pembersihan, plastik]"


In [ ]:
df_keyword.to_excel('keyword_keybert_textrank.xlsx', index = False)

# Evaluasi

In [ ]:
def calculate_evaluation_metrics(df, annotator_name, method_name):
    """
    Menghitung presisi, recall, dan F1-score untuk setiap baris
    berdasarkan kata kunci yang benar (true) dan yang diprediksi (predicted).

    Args:
        df (pd.DataFrame): DataFrame yang berisi kolom kata kunci.
        nama_anotator (str): Nama anotator (contoh: 'sihab').
        nama_metode (str): Nama metode ekstraksi kata kunci (contoh: 'keybert', 'textrank').

    Returns:
        pd.DataFrame: DataFrame dengan kolom tambahan untuk presisi, recall, dan F1-score
                      untuk anotator dan metode yang ditentukan.
    """
    # Pastikan kolom yang dibutuhkan ada
    true_keyword_col = f"keyword_{annotator_name}"
    predicted_keyword_col = f"keyword_{method_name}"

    if true_keyword_col not in df.columns:
        print(f"Error: Column '{true_keyword_col}' not found in the DataFrame.")
        return df
    if predicted_keyword_col not in df.columns:
        print(f"Error: Column '{predicted_keyword_col}' not found in the DataFrame.")
        return df

    for index, row in df.iterrows():
        # Gunakan regular expression untuk mengekstrak kata dari representasi string list
        # Pola ini mencari teks di dalam tanda kutip tunggal (')
        # Pastikan nilai kolom diperlakukan sebagai string sebelum menerapkan re.findall
        keyword_benar_list = re.findall(r"'([^']+)'", str(row[true_keyword_col]))
        keyword_prediksi_list = re.findall(r"'([^']+)'", str(row[predicted_keyword_col]))

        # Konversi daftar kata kunci yang diekstrak menjadi set
        keyword_benar_set = set(keyword_benar_list)
        keyword_prediksi_set = set(keyword_prediksi_list)

        intersection = keyword_benar_set.intersection(keyword_prediksi_set)

        precision = len(intersection) / len(keyword_prediksi_set) if len(keyword_prediksi_set) != 0 else 0
        recall = len(intersection) / len(keyword_benar_set) if len(keyword_benar_set) != 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

        df.at[index, f"presisi_{annotator_name}_{method_name}"] = precision
        df.at[index, f"recall_{annotator_name}_{method_name}"] = recall
        df.at[index, f"f1_score_{annotator_name}_{method_name}"] = f1_score

    return df

In [ ]:
def calculate_mean_evaluation(df, anotator, metode):
    """
    Menghitung rata-rata presisi, recall, dan F1-score untuk setiap anotator
    dan metode, lalu menampilkan hasilnya dalam tabel.

    Args:
        df (pd.DataFrame): DataFrame yang berisi kolom metrik evaluasi.
        anotator (list): Daftar nama anotator (contoh: ['sihab', 'naufal']).
        metode (list): Daftar nama metode (contoh: ['keybert', 'textrank']).
    """
    print("Rata-rata Metrik Evaluasi:")

    for nama_anotator in anotator:
        hasil_anotator = {'Metode': metode}
        presisi_list = []
        recall_list = []
        f1_score_list = []

        for nama_metode in metode:
            kolom_presisi = f"presisi_{nama_anotator}_{nama_metode}"
            kolom_recall = f"recall_{nama_anotator}_{nama_metode}"
            kolom_f1_score = f"f1_score_{nama_anotator}_{nama_metode}"

            # Pastikan kolom ada sebelum menghitung rata-rata
            if kolom_presisi in df.columns and kolom_recall in df.columns and kolom_f1_score in df.columns:
                rata_rata_presisi = df[kolom_presisi].mean()
                rata_rata_recall = df[kolom_recall].mean()
                rata_rata_f1_score = df[kolom_f1_score].mean()

                presisi_list.append(rata_rata_presisi)
                recall_list.append(rata_rata_recall)
                f1_score_list.append(rata_rata_f1_score)
            else:
                print(f"Peringatan: Kolom metrik untuk {nama_anotator} vs {nama_metode} tidak lengkap.")
                presisi_list.append(None)
                recall_list.append(None)
                f1_score_list.append(None)

        hasil_anotator['Presisi'] = presisi_list
        hasil_anotator['Recall'] = recall_list
        hasil_anotator['Skor F1'] = f1_score_list

        df_hasil_anotator = pd.DataFrame(hasil_anotator)
        print(f"\nHasil untuk Anotator: {nama_anotator}")
        print(df_hasil_anotator)

In [ ]:
# Hitung metrik untuk keybert
df = calculate_evaluation_metrics(df_keyword, 'sihab', 'keybert')
df = calculate_evaluation_metrics(df_keyword, 'naufal', 'keybert')

# Hitung metrik untuk textrank
df = calculate_evaluation_metrics(df_keyword, 'sihab', 'textrank')
df = calculate_evaluation_metrics(df_keyword, 'naufal', 'textrank')

# Hitung rata-rata metrik untuk setiap metode
calculate_mean_evaluation(df, ['sihab', 'naufal'], ['keybert', 'textrank'])

Rata-rata Metrik Evaluasi:


<IPython.core.display.Javascript object>


Hasil untuk Anotator: sihab
     Metode   Presisi    Recall   Skor F1
0   keybert  0.266722  0.483000  0.338131
1  textrank  0.307167  0.319333  0.311992


<IPython.core.display.Javascript object>


Hasil untuk Anotator: naufal
     Metode   Presisi  Recall   Skor F1
0   keybert  0.240278   0.406  0.295138
1  textrank  0.261000   0.248  0.253222


Berdasarkan evaluasi terhadap dua metode ekstraksi kata kunci, yaitu KeyBERT dan TextRank, KeyBERT menunjukkan hasil yang lebih baik. Untuk anotator Sihab, KeyBERT mencatat F1-score sebesar 0.338, lebih tinggi dibandingkan TextRank yang hanya mencapai 0.312. Hal yang sama terlihat pada anotator Naufal, di mana KeyBERT memperoleh F1-score 0.295, sedangkan TextRank hanya 0.253. Meskipun presisinya sedikit lebih rendah, KeyBERT unggul dalam recall, yang berarti mampu menangkap lebih banyak kata kunci relevan. Dengan hasil tersebut, KeyBERT dinilai lebih cocok digunakan untuk ekstraksi kata kunci karena lebih mendekati hasil anotasi manual.

# Keyword Extraction KeyBERT

In [ ]:
# Load data dan ekstraksi keyword
df = pd.read_excel('/content/Dataset_Anotasi.xlsx')
df['keluhan_baku'] = df['keluhan_baku'].fillna('')
df['keluhan_baku'] = df['keluhan_baku'].astype(str)
df["keyword_keybert"] = df["keluhan_baku"].apply(lambda x: [kw[0] for kw in kw_model.extract_keywords(x, top_n=10)])

# Menggabungkan semua keyword dari seluruh baris
all_keywords = [keyword for sublist in df["keyword_keybert"] for keyword in sublist]

# Hitung frekuensi kemunculan setiap keyword
keyword_counts = Counter(all_keywords)

# Konversi ke DataFrame dan urutkan
keyword_df = pd.DataFrame(keyword_counts.items(), columns=["keyword", "jumlah"])
keyword_df = keyword_df.sort_values(by="jumlah", ascending=False).reset_index(drop=True)

# Simpan ke Excel
keyword_df.to_excel("/content/Keyword_KeyBERT.xlsx", index=False)

<IPython.core.display.Javascript object>